In [115]:
# from util import *
import torch
from multiprocessing import Pool
import os
from model import WDGRL, AutoEncoder
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kstest
import time
from numba import cuda, float32, float64
import math

In [116]:
print(cuda.gpus)

<Managed Device 0>


In [117]:
d = 32
generator_hidden_dims = [500, 100, 20]
critic_hidden_dims = [100]
wdgrl = WDGRL(input_dim=d, generator_hidden_dims=generator_hidden_dims, critic_hidden_dims=critic_hidden_dims)
index = None
with open("model/wdgrl_models.txt", "r") as f:
    lines = f.readlines()
    for i, line in enumerate(lines):
        words = line[:-1].split("/")
        if words[1] == str(generator_hidden_dims) and words[2] == str(critic_hidden_dims):
            index = i
            break
if index is None:
    print("Model not found")
    exit()
check_point = torch.load(f"model/wdgrl_{index}.pth", map_location=wdgrl.device, weights_only=True)
wdgrl.generator.load_state_dict(check_point['generator_state_dict'])
wdgrl.critic.load_state_dict(check_point['critic_state_dict'])
wdgrl.generator = wdgrl.generator.cpu()

input_dim = generator_hidden_dims[-1]
encoder_hidden_dims = [16, 8, 4, 2]
decoder_hidden_dims = [4, 8, 16, input_dim]
ae = AutoEncoder(input_dim=input_dim, encoder_hidden_dims=encoder_hidden_dims, decoder_hidden_dims=decoder_hidden_dims)
index = None
with open("model/ae_models.txt", "r") as f:
    lines = f.readlines()
    for i, line in enumerate(lines):
        words = line[:-1].split("/")
        if words[1] == str(input_dim) and words[2] == str(encoder_hidden_dims) and words[3] == str(decoder_hidden_dims):
            index = i
            break
if index is None:
    print("Model not found")
    exit()
check_point = torch.load(f"model/ae_{index}.pth", map_location=ae.device, weights_only=True)
ae.load_state_dict(check_point['state_dict'])
ae.net = ae.net.cpu()

In [118]:
def convert_network_to_numpy(model):
    layers = []
    list_layers = []
    ptr = 0
    for name, param in model.named_children():
        temp = dict(param._modules)
        
        for layer_name in temp.values():
            if ('Linear' in str(layer_name)):
                layers.append('Linear')
            elif ('ReLU' in str(layer_name)):
                layers.append('ReLU')
    for name, param in model.named_parameters():
        if (layers[ptr] == 'Linear'):
            if ('weight' in name):
                weight = np.asarray(param.data.cpu())
                print(weight)
                list_layers.append((layers[ptr] + ' ' + 'Weight', weight))
            elif ('bias' in name):
                bias = np.asarray(param.data.cpu()).reshape(-1, 1)
                list_layers.append((layers[ptr] + ' ' + 'Bias', bias))
                ptr += 1

        if (ptr < len(layers) and layers[ptr] == 'ReLU'):
            list_layers.append((layers[ptr], None))
            ptr += 1
    return list_layers

numpy_params = convert_network_to_numpy(wdgrl.generator)
print(numpy_params)

[[-0.1427628   0.06336475 -0.13283369 ...  0.07746582 -0.0131233
  -0.06700886]
 [-0.09125396  0.09734394 -0.03836136 ... -0.00543398 -0.01536715
  -0.18574236]
 [-0.00530715 -0.13076226 -0.11174504 ... -0.1575845  -0.09184723
   0.13326932]
 ...
 [-0.03949018 -0.02248619  0.1237155  ... -0.01993961 -0.0019327
   0.03751336]
 [-0.02166711  0.03713333 -0.07954837 ...  0.0443318  -0.10978504
  -0.15140077]
 [-0.09939715  0.06878612 -0.09405878 ...  0.04489744 -0.01013681
  -0.11121824]]
[[-0.11583654 -0.11675502 -0.04576399 ...  0.03592154 -0.08751635
   0.02787321]
 [-0.09823349 -0.04690997 -0.05323979 ...  0.01594155 -0.0326299
  -0.00829471]
 [-0.00096919 -0.02932019 -0.07357178 ... -0.04478681 -0.02650368
   0.02664188]
 ...
 [-0.01466196 -0.06502712  0.01373101 ... -0.0781096  -0.04929662
  -0.05444728]
 [-0.0645531  -0.01838117 -0.0347773  ...  0.01237289  0.02029328
   0.02667821]
 [-0.04263571  0.03485689 -0.01417724 ...  0.01515914 -0.05302436
   0.05626807]]
[[-0.00909528  0.04

In [119]:
TPB = 32
@cuda.jit
def fast_matmul(A, B, C):
    """
    Perform matrix multiplication of C = A * B using CUDA shared memory.

    Reference: https://stackoverflow.com/a/64198479/13697228 by @RobertCrovella
    """
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = float32(0.)
    for i in range(bpg):
        # Preload data into shared memory
        sA[ty, tx] = 0
        sB[ty, tx] = 0
        if y < A.shape[0] and (tx + i * TPB) < A.shape[1]:
            sA[ty, tx] = A[y, tx + i * TPB]
        if x < B.shape[1] and (ty + i * TPB) < B.shape[0]:
            sB[ty, tx] = B[ty + i * TPB, x]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[ty, j] * sB[j, tx]

        # Wait until all threads finish computing
        cuda.syncthreads()
    if y < C.shape[0] and x < C.shape[1]:
        C[y, x] = tmp

@cuda.jit
def fast_matadd_bias(A, B, C):
    x, y = cuda.grid(2)
    if (x < C.shape[0] and y < C.shape[1]):
        C[x, y] = A[x, y] + B[y, 0]

@cuda.jit
def relu_itv(a, b, X, itv):
    x, y = cuda.grid(2)
    if (x < a.shape[0] and y < a.shape[1]):
        if X[x, y] < 0:
            X[x, y] = 0
            a[x, y] = 0
            b[x, y] = 0
            itv[1] = min(itv[1], -a[x, y] / b[x, y])
        else:
            itv[0] = max(itv[0], a[x, y] / b[x, y])

def matmul(A, B):
    C = cuda.device_array((A.shape[0], B.shape[1]))
    threadsperblock = (TPB, TPB)
    grid_y_max = max(A.shape[0], B.shape[0])
    grid_x_max = max(A.shape[1], B.shape[1])
    blockspergrid_x = math.ceil(grid_x_max / threadsperblock[0])
    blockspergrid_y = math.ceil(grid_y_max / threadsperblock[1])
    blockspergrid = (blockspergrid_x, blockspergrid_y)
    fast_matmul[blockspergrid, threadsperblock](A, B, C)
    return C

def matadd(A, B):
    C = np.zeros((A.shape[0], A.shape[1]), dtype=np.float64)
    C = cuda.to_device(C)
    
    threadsperblock = (TPB, TPB)
    blockspergrid = (int(math.ceil(C.shape[0] / threadsperblock[0])), int(math.ceil(C.shape[1] / threadsperblock[1])))
    fast_matadd_bias[blockspergrid, threadsperblock](A, B, C)
    return C

In [120]:
def intersect(itv1, itv2):
    itv = [max(itv1[0], itv2[0]), min(itv1[1], itv2[1])]
    if itv[0] > itv[1]:
        return None    
    return itv

def solve_linear_inequality(u, v): #u + vz < 0
    if (v > -1e-16 and v < 1e-16):
        if (u <= 0):
            return [-np.inf, np.inf]
        else:
            print('error', u, v)
            return None
    if (v < 0):
        return [-u/v, np.inf]
    return [-np.inf, -u/v]

In [121]:
def linear_weight(a, b, x, weight):
    return matmul(a, weight.T), matmul(b, weight.T), matmul(x, weight.T)

def linear_bias(a, b, x, bias):
    return matadd(a, bias), matadd(b, bias), matadd(x, bias)

def relu(a, b, x, itv):
    threadsperblock = (TPB, TPB)
    blockspergrid = (int(math.ceil(a.shape[0] / threadsperblock[0])), int(math.ceil(a.shape[1] / threadsperblock[1])))
    relu_itv[blockspergrid, threadsperblock](a, b, x, itv)
    return a, b, x, itv

In [122]:
def get_dnn_interval(a, b, x, list_layers):
    a = cuda.to_device(a)
    b = cuda.to_device(b)
    x = cuda.to_device(x)
    itv = np.asarray([-np.inf, np.inf])
    itv = cuda.to_device(itv)
    for name, param in list_layers:
        if name == 'Linear Weight':
            a_g, b_g, x_g = linear_weight(a_g, b_g, x_g, cuda.to_device(param))
        elif name == 'Linear Bias':
            a, b, x = linear_bias(a, b, x, cuda.to_device(param))
        elif name == 'ReLU':
            a, b, x, itv = relu(a, b, x, itv)
    itv = itv.copy_to_host()
    return itv

In [125]:
A = np.full((175, 500), 1, float)
B = np.full((500, 100), 1, float)
print(A.dot(B))
print(matmul(A, B).copy_to_host())

[[500. 500. 500. ... 500. 500. 500.]
 [500. 500. 500. ... 500. 500. 500.]
 [500. 500. 500. ... 500. 500. 500.]
 ...
 [500. 500. 500. ... 500. 500. 500.]
 [500. 500. 500. ... 500. 500. 500.]
 [500. 500. 500. ... 500. 500. 500.]]
[[500. 500. 500. ... 500. 500. 500.]
 [500. 500. 500. ... 500. 500. 500.]
 [500. 500. 500. ... 500. 500. 500.]
 ...
 [500. 500. 500. ... 500. 500. 500.]
 [500. 500. 500. ... 500. 500. 500.]
 [500. 500. 500. ... 500. 500. 500.]]


In [124]:
A = np.full((4, 4), 1, float) # [32 x 48] matrix containing all 3's
B = np.full((4, 4), 1, float) # [48 x 16] matrix containing all 4's
print(A.dot(B))

[[4. 4. 4. 4.]
 [4. 4. 4. 4.]
 [4. 4. 4. 4.]
 [4. 4. 4. 4.]]
